In [1]:
import sys
import os, shutil
from os.path import join
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
sys.path.append('../../')
from LCBDtools.src import argParser

In [3]:
os.path

<module 'posixpath' from '/Users/jfr5989/opt/anaconda3/envs/nirs/lib/python3.7/posixpath.py'>

In [15]:
# parameters

#nirs_dir = "data/NIRS_data_clean/"
nirs_dir = "data/NIRS_data_clean_new files/"
psypy_dir = "data/psychopy"
ex_subs = []
participant_num_len = 4
sample_rate = 7.81250
n_blocks = 3

In [16]:
nirs_session_dirs = [os.path.split(d)[0] for d in glob(
    nirs_dir+"**/*_probe*",
    recursive=True)]

In [17]:
nirs_session_dirs

['data/NIRS_data_clean_new files/1264/1264-P',
 'data/NIRS_data_clean_new files/1264/1264-C',
 'data/NIRS_data_clean_new files/1270/1270-P',
 'data/NIRS_data_clean_new files/1270/1270-C',
 'data/NIRS_data_clean_new files/1267/1267-P',
 'data/NIRS_data_clean_new files/1267/1267-C',
 'data/NIRS_data_clean_new files/1269/1269-C',
 'data/NIRS_data_clean_new files/1269/1269-P',
 'data/NIRS_data_clean_new files/1268/1268-C',
 'data/NIRS_data_clean_new files/1268/1268-P']

In [18]:
def timeconvert_psychopy_to_nirstar(
    sample_rate,
    NSstim1_t,
    PSstim1_t,
    PSevent_t):
    
    NSevent_t = NSstim1_t + (PSevent_t - PSstim1_t) * sample_rate
    return NSevent_t

In [19]:
for ses in tqdm(nirs_session_dirs):
    # should always work, even with multiple-runs. 
    # TODO remember that these exist, where 2 runs are collected
    # for the same dyad of parent/child, for unknown reason. 
    # potentially check for truncated run while parsing this data in other
    # mne processing software
    
    child_sub = ses.strip(nirs_dir).strip("/")[:participant_num_len]
    print(child_sub)
    # looks like it's working!
    
    #if ("1237" in child_sub) or ("1237" in child_sub):
     #   pass
    #else:
     #   continue
    
    # only do this if there's only 1 .evt file in study folder
    og_evt = glob(ses+"/*.evt")
    if len(og_evt) != 1:
        print("Error: found", len(og_evt), "evt files for the given sub / visit.")
        print("Skipping:", ses)
        continue
    else:
        og_evt = og_evt[0]
    
    # open original evt
    f = open(og_evt, 'r')
    line1 = f.readline()
    f.close()
    
    # first time marker is NIRS stim start 1
    NSstim1_t = line1.split('\t')[0]

    
    task_file = glob(join(psypy_dir, child_sub)+ "/*.csv")
    
    # only continue if we have exactly 1 task file found
    if len(task_file) != 1:
        print("Error: found", len(task_file), "task files for the given sub / visit.")
        continue
    else:
        task_file = task_file[0]
      
    # df of psychopy csv output
    df = pd.read_csv(task_file)
    
    stims = []
    
    # iterate through any column that has "intro_txt" and ".started"
    block_i = 0
    for col in df.columns:
        
        # if we're dealing with a legitimate block column
        if ("intro_txt" in col) and (".stopped" in col):

            # then also generate the name of the corresponding stop column
            block_name_str = col.strip("intro_txt").strip(".stopped")
            
            stopcol = "intro_txt"+block_name_str+".started"
            #print(stopcol)
            
            
            # store any non-NaN vals in the start and stop column (times)
            starts = (df[~df[col].isnull()][col].astype(float) + 5).tolist()
            stops = (df[~df[stopcol].isnull()][stopcol].astype(float)).tolist()
            
            if len(starts) != len(stops):
                print("Unequal number of starts and stops. ses:", ses, "block_i", block_i)
                continue
            
            # if there are no starts already entered / outlines has length 1,
            # then we know that the first start in our starts is the same as line1 trigger. 
            # meaning that is the reference point to which the time course of the psychopy
            # file and the NIRS file will be aligned, again using the sample rate
            # or, easily, if block_i == 1.
            
            # append to stims with tuples of (time, evt_stim_col)
            for i in range(len(starts)):
                stims.append((starts[i], block_i))
                stims.append((stops[i], 7))
            
            # if we're in the first block, we know psychopy stim 1 now
            if block_i == 0:
                PSstim1_t = starts[0]
                stims.pop(0)
            
            # convert using first NIRS stim and first Psychopy stim to align
            converted_stims = []
            for (time, val) in stims:
                converted_stims.append((
                    timeconvert_psychopy_to_nirstar(
                        float(sample_rate),
                        float(NSstim1_t),
                        float(PSstim1_t),
                        float(time)),
                    val))
            
            block_i += 1
        
        # if we're not dealing with a legitimate column, we can just keep iterating
        else:
            continue

    output_lines = [line1]
    
    for (stim_time, stim_col) in converted_stims:
        # make bit format stims
        evts = [0]*8
        evts[stim_col]=1
        
        # and write these as lines in correct format for .evt
        line = str(round(stim_time))
        for evt_col in evts:
            line += "\t"
            line += str(evt_col)
        line += "\n"
            
        output_lines.append(line)
    
    # move OG evt to _old.evt
    shutil.move(
        og_evt,
        og_evt.replace(".evt", "_old.evt"))
    
    # write the lines as the new, properly named .evt
    f = open(og_evt, 'w')
    
    for line in output_lines:
        f.write(line)
    
    f.close()

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 63.37it/s]

1264
1264
1270
1270
1267
1267
1269
1269
1268
1268
